# Preprocessing: links in video descriptions

This notebook processes links in video descriptions. Our goal is to extract URLs that may be related to sponsors, hence giving us a way to identify sponsorships in videos.

In [1]:
import findspark
findspark.init()

import re
from pyspark.sql.functions import col, udf, explode, when
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, LongType, StringType, DateType, ArrayType, BooleanType

from pyspark.sql import SparkSession
import pyspark as ps
config = ps.SparkConf().setAll([
    ('spark.network.timeout', '3601s'),
    ('spark.executor.heartbeatInterval', '3600s'),
])
sc = ps.SparkContext('local', '', conf=config)
spark = SparkSession(sc)

In [2]:
USE_SUBDATASETS = True
PATH_METADATAS_SRC = '../data/subdata/yt_metadata_en_sub' if USE_SUBDATASETS else '../data/yt_metadata_en.jsonl'

## Extract links from video descriptions

Firstly, we extract links using regular expressions. We perform a first filtering to remove urls from sites that are generally not related to sponsors such as Youtube, Twitter, Facebook, Wikipedia, Discord, etc...

In [3]:
PATH_METADATAS_URLS_DST = '../data/generated/yt_metadata_en_urls.parquet'

In [4]:
# Load the invalid URLs
PATH_INVALID_URLS = '../data/invalid_urls.csv'

invalid_urls_reg = []
with open(PATH_INVALID_URLS, 'r') as f:
    for line in f:
        invalid_urls_reg.append(fr"(?i)({line.strip()})")

In [5]:
def get_urls(description):
    if description is None:
        return []
        
    url_regex = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"
    urls = []
    for line in description.split("\n"):
        if re.search(url_regex, line):
            urlss = [x.group() for x in re.finditer(url_regex, line)] # Find all urls in the line
            urlss = [x for x in urlss if not any([re.search(reg, x) for reg in invalid_urls_reg])] # Filter out invalid urls
            urls.extend(urlss)
    urls = list(set(urls)) # Remove duplicates
    return urls

# Test the function on basic examples
assert(get_urls('There is no link.') == [])
assert(get_urls('This is not a valid link: www.special.com/') == [])
assert(get_urls('This is a link: https://www.special.com/ and this is another link: https://www.youtube.com/watch?v=2') == ['https://www.special.com/'])
assert(get_urls('This is a link: https://www.special.com/ \n and this is an invalid link: www.special.com/') == ['https://www.special.com/'])

In [6]:
schema = StructType([
    StructField("categories",    StringType(),  True),
    StructField("channel_id",    StringType(),  True),
    StructField("crawl_date",    DateType(),    True),
    StructField("description",   StringType(),  True),
    StructField("dislike_count", DoubleType(),  True), # This field must be specified as a double as it is represented as a floating point number
    StructField("display_id",    StringType(),  True),
    StructField("duration",      IntegerType(), True),
    StructField("like_count",    DoubleType(),  True), # This field must be specified as a double as it is represented as a floating point number
    StructField("tags",          StringType(),  True),
    StructField("title",         StringType(),  True),
    StructField("upload_date",   DateType(),    True),
    StructField("view_count",    DoubleType(),  True)  # This field must be specified as a double as it is represented as a floating point number
])
    
metadatas = spark.read.json(PATH_METADATAS_SRC, schema=schema)

# Cast the dislike_count, like_count and view_count to their respective integer type
metadatas = metadatas \
    .withColumn("dislike_count", metadatas.dislike_count.cast(IntegerType())) \
    .withColumn("like_count", metadatas.like_count.cast(IntegerType())) \
    .withColumn("view_count", metadatas.view_count.cast(LongType()))

# Get urls and count of urls in the description
get_urls_udf = udf(lambda x: get_urls(x), ArrayType(StringType()))
metadatas = metadatas.withColumn('urls', get_urls_udf(col('description')))
len_udf = udf(lambda x: len(x), IntegerType())
metadatas = metadatas.withColumn('urls_count', len_udf(col('urls')))
has_urls_udf = udf(lambda x: x > 0, BooleanType())
metadatas = metadatas.withColumn('has_urls', has_urls_udf(col('urls_count')))

# Drop the description and take only the videos with at least one url
metadatas_urls = metadatas.drop('description')

In [7]:
metadatas_urls.filter(metadatas_urls.has_urls).select('display_id', 'urls', 'urls_count').show(5)

+-----------+--------------------+----------+
| display_id|                urls|urls_count|
+-----------+--------------------+----------+
|VPqJmODeZyk|[https://goo.gl/J...|         1|
|xLYGF-aCEHk|[https://goo.gl/J...|         1|
|TqzfdwSZdRc|[https://goo.gl/J...|         1|
|C-dn3p-ZTrM|[https://goo.gl/J...|         1|
|OP_njme3T84|[https://goo.gl/J...|         1|
+-----------+--------------------+----------+
only showing top 5 rows



In [8]:
metadatas_urls.filter(~metadatas_urls.has_urls).select('display_id', 'urls', 'urls_count').show(5)

+-----------+----+----------+
| display_id|urls|urls_count|
+-----------+----+----------+
|SBqSc91Hn9g|  []|         0|
|UuugEl86ESY|  []|         0|
|oB4c-yvnbjs|  []|         0|
|ZaV-gTCMV8E|  []|         0|
|cGvL7AvMfM0|  []|         0|
+-----------+----+----------+
only showing top 5 rows



Run the two cells below if you want to write the current state of the dataframe to a parquet file.

In [9]:
# Delete the output folder if it already exists
#!rm -f $PATH_METADATAS_URLS_DST # Linux
!PowerShell.exe -Command "Remove-Item -Path $PATH_METADATAS_URLS_DST -Recurse -Force" # Windows

In [10]:
metadatas_urls.write.parquet(PATH_METADATAS_URLS_DST, partitionBy=['has_urls'], mode='overwrite')

## Analyse shortened URLs

In [45]:
regs = {
    'bitly'     : r'https?:\/\/bit\.ly\/[a-zA-Z0-9\-\_]+',
    'google'    : r'https?:\/\/goo\.gl\/[a-zA-Z0-9\-\_]+',
    'tinyurl'   : r'https?:\/\/tinyurl\.com\/[a-zA-Z0-9\-\_]+',
    'adfly'     : r'https?:\/\/adf\.ly\/[a-zA-Z0-9\-\_]+',
    'owly'      : r'https?:\/\/ow\.ly\/[a-zA-Z0-9\-\_]+',
}

In [49]:
for short_name, short_reg in regs.items():
    reg_udf = udf(lambda x: len([y for y in x if re.search(short_reg, y)]), IntegerType())
    metadatas_urls = metadatas_urls.withColumn(short_name + '_count', reg_udf(col('urls')))

#metadatas_urls.select('display_id', 'urls', 'bitly_count', 'google_count', 'tinyurl_count', 'adfly_count').show(5)
metadatas_urls.filter(metadatas_urls.google_count > 0).select('urls').show(5, False)


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|urls                                                                                                                                                                                                                                                                                                                                                                                                  |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [48]:
short_urls_cnt = metadatas_urls.select('bitly_count', 'google_count', 'tinyurl_count', 'adfly_count', 'owly_count') \
    .agg({'bitly_count': 'sum', 'google_count': 'sum', 'tinyurl_count': 'sum', 'adfly_count': 'sum', 'owly_count': 'sum'}) \
    .collect()[0] \
    .asDict()

short_urls_cnt

{'sum(bitly_count)': 2783,
 'sum(adfly_count)': 9,
 'sum(owly_count)': 11,
 'sum(tinyurl_count)': 163,
 'sum(google_count)': 2373}

## Resolve Bitly URLs

To further enrich our dataset, we write in a separate file all the Bitly URLs found in the dataset. We will then use various tools such as the Bitly API, or simple HTTP requests to resolve the URLs.

In [28]:
PATH_BITLY_URLS = '../data/generated/bitly_urls.tsv'

In [12]:
# Delete the output folder if it already exists
#!rm -f $PATH_BITLY_URLS # Linux
!PowerShell.exe -Command "Remove-Item -Path $PATH_BITLY_URLS -Recurse -Force" # Windows

In [13]:
bitly_urls = metadatas_urls.select('display_id', explode('urls').alias('url'))
bitly_urls = bitly_urls.filter(bitly_urls.url.like('%bit.ly%')).select('url').distinct()

bitly_urls.write.csv(PATH_BITLY_URLS, sep='\t', header=False)

We now assume that the **Bitly URLs have been resolved** and we can replace them by their corresponding URLs.

In [58]:
PATH_BITLY_RESOLVED_URLS = '../data/generated/bitly_resolved_urls.tsv'

In [59]:
schema = StructType([
    StructField("url",          StringType(), True),
    StructField("resolved_url", StringType(), True),
])
resolved_urls = spark.read.csv(PATH_BITLY_RESOLVED_URLS, sep='\t', header=False, schema=schema)
resolved_urls.show(5)

+--------------------+--------------------+
|                 url|        resolved_url|
+--------------------+--------------------+
|http://bit.ly/2Cz...|http://www.musicn...|
|http://bit.ly/2Cz...|                None|
|https://bit.ly/2w...|https://www.audio...|
|http://bit.ly/2Lq...|https://www.audio...|
|http://bit.ly/2rw...|https://www.audio...|
+--------------------+--------------------+
only showing top 5 rows



In [60]:
metadatas_resolved_urls = metadatas_urls.withColumn('url', explode('urls')).join(resolved_urls, 'url', 'left')
metadatas_resolved_urls.filter(metadatas_resolved_urls.display_id == 'FJMKM_TYLy8').select('display_id', 'url', 'resolved_url').show(5, True)

+-----------+--------------------+--------------------+
| display_id|                 url|        resolved_url|
+-----------+--------------------+--------------------+
|FJMKM_TYLy8|http://NCS.lnk.to...|                null|
|FJMKM_TYLy8|http://bit.ly/ALL...|https://www.youtu...|
|FJMKM_TYLy8|http://bit.ly/Dre...|https://soundclou...|
|FJMKM_TYLy8|http://bit.ly/NCS...|http://www.youtub...|
|FJMKM_TYLy8|http://bit.ly/NCS...|http://www.youtub...|
+-----------+--------------------+--------------------+
only showing top 5 rows



In [61]:
metadatas_resolved_urls = metadatas_resolved_urls \
    .withColumn('url', \
        when(metadatas_resolved_urls.resolved_url.isNotNull(), 
            metadatas_resolved_urls.resolved_url)
        .otherwise(
            metadatas_resolved_urls.url)) \
    .drop('resolved_url')
metadatas_resolved_urls.filter(metadatas_resolved_urls.display_id == 'FJMKM_TYLy8').select('display_id', 'url').show(5, True)

+-----------+--------------------+
| display_id|                 url|
+-----------+--------------------+
|FJMKM_TYLy8|http://NCS.lnk.to...|
|FJMKM_TYLy8|https://www.youtu...|
|FJMKM_TYLy8|https://soundclou...|
|FJMKM_TYLy8|http://www.youtub...|
|FJMKM_TYLy8|http://www.youtub...|
+-----------+--------------------+
only showing top 5 rows

